In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyswarms as ps
from sklearn.preprocessing import StandardScaler

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [5]:
data = pd.read_csv(r"file_path.csv")

# CHANGE THE FILE PATH!
# Extract features and target
X = data.iloc[:, :-1].values  # the Features
y = data.iloc[:, -1].values   # Target
print("donee")

donee


In [5]:
#added
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Network architecture parameters
n_inputs = X.shape[1]
n_hidden = 20
n_classes = len(np.unique(y))
num_samples = X.shape[0]

In [6]:
def logits_function(p):
    """ Calculate roll-back the weights and biases """
    #edited
    W1_end = n_inputs * n_hidden
    b1_end = W1_end + n_hidden
    W2_end = b1_end + n_hidden * n_classes
    
    W1 = p[:W1_end].reshape((n_inputs, n_hidden))
    b1 = p[W1_end:b1_end].reshape((n_hidden,))
    W2 = p[b1_end:W2_end].reshape((n_hidden, n_classes))
    b2 = p[W2_end:].reshape((n_classes,))
    
    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2  # Pre-activation in Layer 2
    return logits


In [7]:
# Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """

    logits = logits_function(params)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [8]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    #edited
    n_particles = x.shape[0]
    loss = np.zeros(n_particles)
    
    for i in range(n_particles):
        logits = logits_function(x[i])
        y_pred = np.argmax(logits, axis=1)
        loss[i] = np.mean((y_pred - y) ** 2)
    
    return loss

In [9]:
%%time
# Initialize swarm
#edited cuz higher accuracy
options = {'c1': 0.6, 'c2': 0.4, 'w':0.8}
#options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
# Call instance of PSO

dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)
# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

2024-05-18 00:37:47,901 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.6, 'c2': 0.4, 'w': 0.8}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|1000/1000, best_cost=0.134
2024-05-18 00:38:37,005 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.13361344537815126, best pos: [ 1.92979059  0.96089478 -0.62751143  2.42742664  0.21502578  1.91182621
  1.7990874   0.49831577  0.61710886  1.57870634  2.59794742  0.92987451
  1.00063124  2.03105391  1.11941467  1.26020228  1.27634105  2.48740745
  2.33265249  0.6336027   0.88834534  2.55105064  1.33708542  1.4708917
  1.093855    0.71322119 -1.43602411  1.49916656  1.74628358 -0.78386571
  0.93732691  1.14659392  0.88720549  1.50277252 -0.86872051  1.41805894
  1.77497787  1.80360886  0.61970869  0.39436206  1.46143389  0.06612377
  1.7718036   2.47801772  2.58827435  0.44026289  1.84152347  1.45172466
 -0.16213305  0.42327081  1.2735766  -0.324

CPU times: total: 4min 50s
Wall time: 49.1 s


In [10]:
def predict(pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    logits = logits_function(pos)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [11]:
# Evaluate the model
accuracy = (predict(pos) == y).mean()
print(f'Model accuracy: {accuracy}')

Model accuracy: 0.8663865546218488
